# Entropia {#sec-entropy}

**Prerequisiti**

**Concetti e competenze chiave**

- Comprendere il concetto di entropia.
- Comprendere il concetto di divervenza di Kullback-Leibler ($D_{\text{KL}}$).
- Calcolare la divergenza $D_{\text{KL}}$ dall'entropia;

**Preparazione del Notebook**

In [10]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.special import kl_div
import arviz as az
import math
import warnings
warnings.filterwarnings("ignore")

In [11]:
%config InlineBackend.figure_format = 'retina'
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")

## Introduzione {.unnumbered}

Nel campo della statistica bayesiana, è fondamentale confrontare diversi modelli predittivi per determinare quale si adatta meglio ai dati disponibili. In questo capitolo, esploreremo il concetto di entropia, una misura essenziale per quantificare l'incertezza nelle distribuzioni di probabilità. L'entropia di una variabile casuale indica il grado medio della sua imprevedibilità.

Analizzeremo anche come l'entropia può essere utilizzata per valutare la "distanza" tra un modello teorico e i dati osservati, introducendo la divergenza di Kullback-Leibler ($D_{\text{KL}}$). Questa misura quantifica le discrepanze tra due distribuzioni di probabilità, fornendo un indicatore di quanto accuratamente un modello rappresenti le osservazioni empiriche.

Nel capitolo successivo, presenteremo la tecnica della Validazione Incrociata Leave-One-Out (LOO-CV), che viene utilizzata per calcolare un'approssimazione della divergenza $D_{\text{KL}}$. Questa tecnica consente di stimare quanto bene un modello generalizzi i dati, offrendo un'ulteriore metrica per la valutazione dei modelli statistici.

## La Generalizzabilità dei Modelli e il Metodo Scientifico

La generalizzabilità dei modelli è un concetto fondamentale nella scienza e uno dei pilastri del metodo scientifico. Questo concetto si riferisce alla capacità di un modello di essere applicato con successo e di produrre risultati accurati al di fuori del contesto specifico o del set di dati per cui è stato inizialmente sviluppato o testato. In altre parole, il valore scientifico di un modello dipende fortemente dalla sua capacità di generalizzare a nuovi dati.

Nella pratica, la generalizzabilità di un modello può essere compromessa da due problemi principali: il sotto-adattamento e il sovra-adattamento. 

- **Sotto-adattamento**: Si verifica quando un modello è troppo semplice per rappresentare adeguatamente la complessità dei dati. Un modello sotto-adattato non riesce a catturare le relazioni essenziali nei dati, portando a prestazioni scadenti sia sul set di dati di addestramento che su nuovi dati. Questo limita gravemente la sua utilità in applicazioni pratiche.

- **Sovra-adattamento**: Si manifesta quando un modello è eccessivamente complesso e si adatta troppo fedelmente al rumore o alle particolarità specifiche del set di dati di addestramento. Un modello sovra-adattato può mostrare ottime prestazioni sui dati di addestramento, ma fallisce nel generalizzare a nuovi dati, riducendo la sua efficacia predittiva.

L'approccio bayesiano alla modellazione offre un modo efficace per bilanciare la complessità del modello con l'adattamento ai dati. Come descritto da @McElreath_rethinking, la selezione del modello è un processo che richiede un equilibrio tra la semplicità del modello e la sua capacità di rappresentare accuratamente la realtà dei dati.

Una pratica comune nella scelta tra modelli alternativi si basa sul principio del rasoio di Ockham, che favorisce le spiegazioni più semplici quando ci sono più teorie equivalenti per un fenomeno. Tuttavia, questo principio da solo non è sufficiente; è fondamentale che il modello scelto descriva accuratamente i dati.

Tradizionalmente, la selezione dei modelli è stata spesso basata sull'uso dei valori-p, ma, come evidenziato da @McElreath_rethinking, questo approccio presenta numerosi problemi e manca di una giustificazione teorica solida. 

Un metodo più robusto e teoricamente fondato è l'uso della divergenza di Kullback-Leibler, che misura quanto un modello riesca ad approssimare efficacemente la distribuzione reale dei dati. Questa misura fornisce una stima quantitativa della precisione del modello nel rappresentare il processo generativo sottostante.

Questo capitolo introduce il concetto di entropia, che è essenziale per comprendere la divergenza di Kullback-Leibler, e le sue applicazioni nella selezione dei modelli, tema che verrà approfondito nel prossimo capitolo.

## Cos'è l'Entropia dell'Informazione?

L'entropia dell'informazione, introdotta da Claude Shannon, è un concetto fondamentale della teoria dell'informazione. Questa grandezza matematica misura l'incertezza o la sorpresa associata alla ricezione di un messaggio. In termini semplici, quanto più un evento è improbabile, tanto maggiore sarà la sorpresa associata alla sua osservazione.

### Definizione di Sorpresa e Entropia

La sorpresa di un evento, determinata dalla sua probabilità $p$, è calcolata utilizzando la formula:

$$ H(p) = -\log_2(p) = \log_2 \left(\frac{1}{p}\right). $$

L'uso del logaritmo in base 2 in questa formula si basa su alcune considerazioni fondamentali:

- **Unità di misura**: La base 2 è utilizzata perché l'informazione viene misurata in "bit". Un bit rappresenta la quantità di informazione derivante da una scelta binaria (ad esempio, il risultato di un lancio di moneta). Ad esempio, l'entropia associata al lancio di una moneta bilanciata è pari a 1 bit.
- **Scala logaritmica**: La scala logaritmica rappresenta più accuratamente la percezione umana dell'informazione e della sorpresa. Ad esempio, eventi con probabilità molto basse hanno un impatto informativo significativamente maggiore rispetto a piccoli cambiamenti nelle probabilità di eventi più probabili.

### Calcolo dell'Entropia

Per capire meglio il concetto di entropia, consideriamo alcuni esempi pratici. La funzione per calcolare l'entropia di un evento con probabilità $p$ è la seguente:

In [6]:
def calcola_entropia(p):
    if p == 0 or p == 1:
        return 0  # Non c'è incertezza se l'evento è certo o impossibile
    else:
        return -p * math.log2(p)


# Esempi di probabilità
probabilità = [0.0, 0.1, 0.5, 0.9, 1.0]

# Calcolo dell'entropia per ciascuna probabilità
entropie = {p: calcola_entropia(p) for p in probabilità}

print(entropie)

{0.0: 0, 0.1: 0.33219280948873625, 0.5: 0.5, 0.9: 0.13680278410054494, 1.0: 0}


L'output di questo script mostra che l'entropia è massima per eventi con probabilità intermedia (0.5) e minima (zero) per eventi certi (probabilità 1) o impossibili (probabilità 0). Questo riflette il fatto che l'incertezza è massima quando tutti gli esiti sono ugualmente probabili e minima quando l'esito è noto con certezza.

### Caratteristiche dell'Entropia

- **Massima Entropia**: L'entropia raggiunge il suo valore massimo quando tutti gli esiti possibili di un evento hanno la stessa probabilità di verificarsi. Questa condizione rappresenta il massimo grado di imprevedibilità, poiché non esistono indizi che possano aiutare a prevedere quale esito si verificherà.
  
- **Minima Entropia**: L'entropia è minima quando l'esito di un evento è completamente certo (probabilità di 1) o impossibile (probabilità di 0). In tali casi, non c'è alcuna incertezza o sorpresa, e quindi non c'è alcuna informazione da guadagnare osservando l'evento.

### Additività dell'Entropia per Eventi Indipendenti

L'entropia ha una proprietà di additività nel caso di eventi indipendenti. Se due o più eventi indipendenti si verificano, l'entropia totale associata alla loro combinazione è uguale alla somma delle entropie di ciascun evento preso singolarmente. Questa proprietà deriva dal fatto che i logaritmi sono additivi, permettendo così di sommare le entropie individuali per ottenere l'entropia complessiva.

### Entropia di Variabili Casuali

L'entropia può essere estesa dal concetto di singolo evento a quello di variabile casuale. In questo contesto, l'entropia misura l'incertezza complessiva associata alla realizzazione di una variabile casuale, che può essere discreta o continua.

#### Entropia di una Variabile Casuale Discreta

Per una variabile casuale discreta $X$ con un insieme di possibili valori $x_1, x_2, \ldots, x_n$ e una funzione di massa di probabilità $p(x) = \Pr\{X = x\}$, l'entropia $H(X)$ è definita come:

$$ H(X) = -\sum_{x \in X} p(x) \log_2 p(x). $$

Questa formula rappresenta la somma pesata delle sorprese di ciascun possibile esito. Il segno negativo è necessario perché i logaritmi delle probabilità, essendo numeri minori di 1, sono negativi. Il segno negativo inverte questi valori, trasformandoli in quantità positive che rappresentano correttamente l'informazione o la sorpresa.

In sintesi, l'entropia $H(X)$ misura l'incertezza complessiva di una variabile casuale discreta, considerando le probabilità di tutti i suoi possibili esiti. Ogni termine della somma $-p(x) \log_2(p(x))$ rappresenta la quantità di sorpresa o informazione associata a ciascun esito, ponderata dalla probabilità di quell'esito.

::: {#exm-}

Sia $X$ una variabile casuale discreta che può assumere i valori $a, b, c,$ e $d$ con una distribuzione di probabilità di massa $p(a) = \frac{1}{2}$, $p(b) = \frac{1}{4}$, $p(c) = \frac{1}{8}$, e $p(d) = \frac{1}{8}$, rispettivamente. L'entropia di $X$, che misura l'incertezza associata alla distribuzione di probabilità, è calcolata come:

$$
H(X) = -\left(\frac{1}{2} \log_2 \frac{1}{2} + \frac{1}{4} \log_2 \frac{1}{4} + \frac{1}{8} \log_2 \frac{1}{8} + \frac{1}{8} \log_2 \frac{1}{8}\right).
$$

Calcolando i singoli termini, otteniamo:

$$
H(X) = -\left(\frac{1}{2} \cdot (-1) + \frac{1}{4} \cdot (-2) + \frac{1}{8} \cdot (-3) + \frac{1}{8} \cdot (-3)\right) = \frac{7}{4} \text{ bits}.
$$

È importante notare che l'entropia $H(X)$ dipende esclusivamente dalla distribuzione di probabilità dei valori di $X$ e non dai valori stessi.

:::

#### Entropia di una Variabile Casuale Continua

Nel caso delle variabili casuali continue, il concetto di entropia viene generalizzato sostituendo la somma con un integrale. Questo è necessario perché le variabili continue possono assumere un numero infinito di valori all'interno di un intervallo.

Per una variabile casuale continua $X$ con una funzione di densità di probabilità $p(x)$, l'entropia (nota anche come entropia differenziale) è definita dalla seguente formula:

$$ H(X) = -\int p(x) \log_2(p(x)) \, dx, $$

dove:

- $p(x)$ è la funzione di densità di probabilità di $X$,
- l'integrale è calcolato su tutto il dominio di $X$.

L'entropia di una variabile casuale continua fornisce una misura dell'incertezza o della sorpresa associata alla distribuzione della variabile. Come nel caso discreto, l'entropia continua quantifica l'incertezza associata a $X$. Una PDF molto concentrata (ad esempio, una distribuzione con picchi stretti) implica bassa entropia, poiché l'evento è più prevedibile. Una PDF distribuita uniformemente implica alta entropia, poiché l'evento è meno prevedibile.

Il segno negativo assicura che l'entropia sia una quantità positiva, in quanto $\log_2(p(x))$ è negativo per $p(x)$ compreso tra 0 e 1.

### Applicazioni Psicologiche

L'entropia dell'informazione trova applicazioni anche in psicologia, per esempio nello studio dell'effetto della sorpresa sull'umore. La sorpresa, o entropia, è stata documentata sia in laboratorio che in contesti naturali come un fattore significativo che influenza le emozioni.

Ad esempio, @spector1956expectations osservò l'effetto della probabilità a priori sulla soddisfazione dei soggetti in risposta a una promozione lavorativa. I risultati indicano che gli esiti meno probabili a priori (e quindi più sorprendenti quando si verificano) hanno un impatto maggiore sull'umore. In altre parole, quando un evento inatteso e sorprendente si verifica, esso tende a influenzare l'umore in modo più forte rispetto a eventi previsti e probabili.

## La Codifica Huffman

La codifica Huffman è un metodo utilizzato per rappresentare gli esiti di una variabile casuale in un formato binario, riducendo al minimo l'incertezza e ottimizzando la lunghezza del codice necessario per descrivere questi esiti. Questo algoritmo crea una rappresentazione binaria che permette di comprimere i dati senza perdita di informazioni, utilizzando una codifica efficiente basata sulla frequenza o probabilità dei simboli.

Nella *codifica Huffman*, i simboli più frequenti sono rappresentati da codici binari più brevi, mentre i simboli meno frequenti sono rappresentati da codici più lunghi. Questo approccio assicura che la lunghezza media del codice per rappresentare una sequenza di simboli sia la più corta possibile, ottimizzando così lo spazio necessario per la memorizzazione o la trasmissione dei dati. In questo modo, la codifica Huffman riesce a ridurre la quantità di bit necessari per codificare una variabile casuale, rispettando il principio della teoria dell'informazione che lega la probabilità di un simbolo alla lunghezza del codice a esso associato.

### Come Funziona la Codifica Huffman

La codifica Huffman è un esempio di **codifica prefissa**, il che significa che nessun codice assegnato a un simbolo è prefisso del codice assegnato a un altro simbolo. Questa proprietà garantisce che la decodifica dei bit sia univoca e senza ambiguità.

Ecco i passaggi per creare una codifica Huffman:

1. **Creare una lista di simboli e le loro frequenze o probabilità**: Si inizia con un insieme di simboli, ognuno associato alla sua frequenza o probabilità di apparizione.

2. **Costruire un albero binario**:
   - Ogni simbolo viene rappresentato come un nodo foglia in un albero binario, con la sua frequenza come "peso" del nodo.
   - Si trovano i due nodi con i pesi minori e si uniscono per formare un nuovo nodo padre. Il peso del nuovo nodo padre è la somma dei pesi dei due nodi figli.
   - Si ripete il passaggio precedente fino a quando rimane un solo nodo, che diventa la radice dell'albero. Questo processo costruisce l'albero di Huffman.

3. **Assegnare i codici**:
   - A partire dalla radice dell'albero, si assegnano 0 e 1 ai rami dell'albero. Generalmente, si assegna "0" a un ramo e "1" all'altro.
   - Il codice di ciascun simbolo è la sequenza di bit che si ottiene attraversando l'albero dalla radice alla foglia corrispondente al simbolo.

### Esempio di Codifica Huffman

Consideriamo un esempio semplice con quattro simboli: $A, B, C,$ e $D$ con le seguenti probabilità:

- $p(A) = 0.4$
- $p(B) = 0.3$
- $p(C) = 0.2$
- $p(D) = 0.1$

#### Passaggio 1: Costruire l'Albero

1. **Lista iniziale dei nodi foglia con le loro probabilità**:

   - $A: 0.4$
   - $B: 0.3$
   - $C: 0.2$
   - $D: 0.1$

2. **Unire i due nodi con le probabilità minori (C e D)**:

   - Creiamo un nodo padre con probabilità $0.2 + 0.1 = 0.3$.
   
3. **Aggiornare la lista e ripetere**:

   - Nuova lista: $A: 0.4, B: 0.3, (CD): 0.3$
   - Unire $B$ e $CD$: Creiamo un nodo padre con probabilità $0.3 + 0.3 = 0.6$.
   
4. **Unire i due nodi rimanenti (A e (BCD))**:

   - Creiamo un nodo padre con probabilità $0.4 + 0.6 = 1.0$, che diventa la radice dell'albero.

L'albero risultante ha questa struttura:

```
          Root (1.0)
         /         \
       A (0.4)   Internal (0.6)
                 /          \
               B (0.3)    Internal (0.3)
                          /         \
                        C (0.2)     D (0.1)
```

#### Passaggio 2: Assegnare i Codici

- Dal nodo radice:
  - Per scendere a $A$, assegniamo "0".
  - Per scendere all'altro ramo (contenente $B, C,$ e $D$), assegniamo "1".
- Continuando l'assegnazione lungo ogni ramo:
  - $A$ = "0"
  - $B$ = "10"
  - $C$ = "110"
  - $D$ = "111"

#### Risultato della Codifica Huffman

Il risultato finale assegna i seguenti codici binari ai simboli:

- $A$ = "0"
- $B$ = "10"
- $C$ = "110"
- $D$ = "111"

Questa distribuzione di codici è ottimale in termini di lunghezza media del codice, data la distribuzione di probabilità dei simboli.

### Perché la Codifica Huffman è Efficiente

La codifica Huffman è efficiente perché:

- **Minimizza la lunghezza media del codice**: Assegna codici più corti ai simboli più frequenti e codici più lunghi a quelli meno frequenti, riducendo così la lunghezza complessiva del messaggio codificato.
- **È una codifica prefissa**: Nessun codice è prefisso di un altro, il che elimina le ambiguità durante la decodifica. Questo significa che, quando si legge un flusso di bit codificati, è sempre possibile identificare in modo univoco quale simbolo rappresentano.

In conclusione, la codifica Huffman è un metodo di compressione dei dati che utilizza le proprietà della probabilità e della teoria dell'informazione per creare una rappresentazione binaria ottimizzata, riducendo così la quantità di dati necessari per rappresentare una sequenza di simboli.

## L'Entropia come Lunghezza Media del Codice Binario

L'entropia, in termini di teoria dell'informazione, rappresenta la quantità media di informazione necessaria per descrivere gli esiti di una variabile casuale. In altre parole, può essere interpretata come la lunghezza media del codice binario utilizzato per rappresentare questi esiti, tenendo conto delle loro probabilità.

Consideriamo una variabile casuale discreta $X$ che può assumere quattro valori: $A, B, C,$ e $D$, con le seguenti probabilità:

- $p(A) = 0.4$
- $p(B) = 0.3$
- $p(C) = 0.2$
- $p(D) = 0.1$

Utilizzando la codifica Huffman per questa variabile casuale, otteniamo i seguenti codici binari:

- $A$ = "0"
- $B$ = "10"
- $C$ = "110"
- $D$ = "111"

La lunghezza media del codice in bit può essere calcolata come segue:

$$
\begin{align}
\text{Lunghezza media} &= p(A) \times \text{lunghezza di } A + p(B) \times \text{lunghezza di } B\notag \\
&\quad + p(C) \times \text{lunghezza di } C\notag + p(D) \times \text{lunghezza di } D\notag.
\end{align}
$$

Sostituendo i valori delle probabilità e le lunghezze dei codici:

$$
\begin{align}
\text{Lunghezza media} &= (0.4 \times 1) + (0.3 \times 2) + (0.2 \times 3) + (0.1 \times 3)\notag\\
&= 0.4 + 0.6 + 0.6 + 0.3 = 1.9 \text{ bit}.\notag
\end{align}
$$

Calcoliamo ora l'entropia $H(X)$ della variabile casuale $X$:

$$
\begin{align}
H(X) &= -\sum p(x) \log_2 p(x) \notag\\
     &= -(0.4 \log_2 0.4 + 0.3 \log_2 0.3 + 0.2 \log_2 0.2 + 0.1 \log_2 0.1)\notag\\ 
     &= 1.8465 \text{ bit}.
\end{align}
$$

In questo esempio, l'entropia $H(X) = 1.8465$ bit rappresenta la lunghezza media minima teorica del codice binario necessario per rappresentare la variabile casuale $X$. La lunghezza media del codice Huffman che abbiamo calcolato è $1.9$ bit, che è molto vicina all'entropia. Questo dimostra che la codifica Huffman è quasi ottimale, avvicinandosi molto al limite inferiore determinato dall'entropia. Pertanto, l'entropia misura effettivamente la lunghezza media minima del codice binario necessaria per descrivere la distribuzione di una variabile casuale.

## L'Entropia Relativa

Una volta compresa la relazione tra l'entropia e la lunghezza media minima del codice binario necessario per descrivere la distribuzione di una variabile casuale, possiamo introdurre il concetto di **entropia relativa**, conosciuta anche come **divergenza di Kullback-Leibler** (KL-divergence). L'entropia relativa è una misura della "distanza" tra due distribuzioni di probabilità, che quantifica quanto una distribuzione si discosta da un'altra.

### Concetto di Divergenza di Kullback-Leibler

La divergenza di Kullback-Leibler, denotata come $D(P \parallel Q)$, misura l'inefficienza di assumere che la distribuzione dei dati sia $Q$ quando, in realtà, la distribuzione vera è $P$. In termini di teoria dell'informazione, $D(P \parallel Q)$ indica quanto è meno efficiente, in termini di lunghezza media del codice, utilizzare una distribuzione errata $Q$ rispetto alla distribuzione corretta $P$.

Se conoscessimo la vera distribuzione $P$ di una variabile casuale, potremmo costruire un codice ottimale con una lunghezza media pari all'entropia $H(P)$. Tuttavia, se utilizzassimo un codice basato su una distribuzione diversa $Q$, la lunghezza media necessaria per descrivere la variabile casuale sarebbe maggiore. Precisamente, questa lunghezza media del codice sarebbe $H(P) + D(P \parallel Q)$ bit. L'incremento rappresentato da $D(P \parallel Q)$ riflette l'inefficienza aggiuntiva derivante dall'utilizzo della distribuzione $Q$ invece della distribuzione vera $P$.

### Definizione Matematica della Divergenza di Kullback-Leibler

Matematicamente, la divergenza di Kullback-Leibler è definita come:

$$
D(P \parallel Q) = \sum_{x} p(x) \log_2 \left( \frac{p(x)}{q(x)} \right),
$$

dove:

- $p(x)$ è la probabilità associata al valore $x$ secondo la distribuzione vera $P$.
- $q(x)$ è la probabilità associata al valore $x$ secondo la distribuzione approssimata $Q$.

Questa formula rappresenta la somma, pesata dalle probabilità $p(x)$, dei logaritmi del rapporto tra le probabilità delle due distribuzioni.

### Entropia, Entropia Incrociata e Divergenza KL

Per chiarire ulteriormente il concetto, consideriamo due misure fondamentali: l'entropia di Shannon e l'entropia incrociata.

1. **Entropia di Shannon**: Se $X$ è una variabile casuale che segue la distribuzione $P$, l'entropia $H(P)$ misura l'incertezza intrinseca di $P$ ed è definita come:
   
   $$
   H(P) = -\sum_x p(x) \log_2(p(x)).
   $$

2. **Entropia incrociata**: L'entropia incrociata $H(P, Q)$ misura l'incertezza media quando si usa la distribuzione $Q$ per descrivere la variabile casuale $X$ distribuita secondo $P$:
   
   $$
   H(P, Q) = -\sum_x p(x) \log_2(q(x)).
   $$

   Questa misura rappresenta la sorpresa media se si assume che i dati siano distribuiti secondo $Q$ quando in realtà seguono $P$.

3. **Divergenza KL**: La divergenza di Kullback-Leibler può essere vista come la differenza tra l'entropia di $P$ e l'entropia incrociata tra $P$ e $Q$:
   
   $$
   D_{\text{KL}}(P \parallel Q) = H(P, Q) - H(P).
   $$
   
   Alternativamente, si può esprimere come:
   
   $$
   D_{\text{KL}}(P \parallel Q) = \sum_x p(x) \log_2 \left(\frac{p(x)}{q(x)}\right).
   $$

### Interpretazione della Divergenza KL

La divergenza $D_{\text{KL}}(P \parallel Q)$ rappresenta il "costo" aggiuntivo di sorpresa o inefficienza quando si utilizza la distribuzione $Q$ per modellare i dati che in realtà seguono la distribuzione $P$. Questo "costo" è espresso in bit e rappresenta l'informazione che viene "persa" quando $Q$ è usata al posto di $P$.

È importante notare che la divergenza di Kullback-Leibler non è simmetrica, il che significa che $D(P \parallel Q) \neq D(Q \parallel P)$. Nonostante non sia una vera distanza matematica, fornisce un'indicazione quantitativa di quanto una distribuzione si discosti dall'altra in termini di inefficienza di codifica.

In sintesi, l'entropia relativa o divergenza di Kullback-Leibler è una misura chiave nella teoria dell'informazione. Essa valuta l'efficacia di un modello probabilistico confrontando una distribuzione teorica con la distribuzione vera dei dati. La divergenza KL fornisce un quadro per quantificare la perdita di informazione e l'incremento di incertezza quando si utilizza una distribuzione approssimativa per descrivere dati reali.

::: {.callout-note}

Un commento sulla notazione. Le lettere maiuscole $P$ e $Q$ sono utilizzate per indicare le distribuzioni di probabilità e lettere minuscole $p(x)$ e $q(x)$ per le funzioni di probabilità specifiche dei valori discreti.

:::

::: {#exm-}

Per fare un esempio, supponiamo che $P$ e $Q$ siano due distribuzioni di probabilità su un insieme finito di possibili esiti, ad esempio {0, 1, 2}. Per semplicità, consideriamo che $P$ e $Q$ siano definite come segue:

- $P$ è la distribuzione "vera": $P = [0.1, 0.6, 0.3]$;
- $Q$ è una distribuzione alternativa che usiamo per la stima: $Q = [0.2, 0.5, 0.3]$.

In [12]:
# Definizione delle distribuzioni
P = np.array([0.1, 0.6, 0.3])
Q = np.array([0.2, 0.5, 0.3])

# Calcolo della divergenza KL da P a Q
KL_divergence = np.sum(kl_div(P, Q))

print(f"Divergenza KL da P a Q: {KL_divergence:.4f}")

Divergenza KL da P a Q: 0.0401


Nel codice precedente, `kl_div(P, Q)` calcola la divergenza $D_{\text{KL}}$ elemento per elemento dell'array. Essa calcola $\sum_x p(x) \log \left(\frac{p(x)}{q(x)}\right)$ per ogni esito $x$, che è esattamente il termine $p(x) \log \left(\frac{p(x)}{q(x)}\right)$ descritto nella formula della divergenza $D_{\text{KL}}$. Utilizziamo poi `np.sum` per sommare tutti i contributi individuali e ottenere il valore totale della divergenza $D_{\text{KL}}$.

Questo esempio fornisce un calcolo diretto della divergenza $D_{\text{KL}}$ tra due distribuzioni, mostrando come una distribuzione $Q$ possa essere inadeguata nel modellare una distribuzione $P$, con un focus sul "costo" di sorpresa per ogni esito.
:::

::: {#exm-}
In un due altri esempi, rendiamo via via $Q$ più diverso da $P$. Notiamo come la divergenza $D_{\text{KL}}$ aumenta.

In [13]:
P = np.array([0.1, 0.6, 0.3])
Q = np.array([0.35, 0.3, 0.35])
KL_divergence = np.sum(kl_div(P, Q))
print(f"Divergenza KL da P a Q: {KL_divergence:.4f}")

Divergenza KL da P a Q: 0.2444


In [14]:
P = np.array([0.1, 0.6, 0.3])
Q = np.array([0.6, 0.3, 0.1])
KL_divergence = np.sum(kl_div(P, Q))
print(f"Divergenza KL da P a Q: {KL_divergence:.4f}")

Divergenza KL da P a Q: 0.5663


:::

## Applicazione della Divergenza $D_{\text{KL}}$ nella Selezione di Modelli

La divergenza di Kullback-Leibler, $D_{\text{KL}}$, è uno strumento fondamentale nella selezione dei modelli statistici. L'obiettivo è identificare il modello $Q$ che minimizza $D_{\text{KL}}(P \parallel Q)$, riducendo al minimo la differenza tra l'entropia della distribuzione vera $P$ e l'entropia incrociata tra $P$ e $Q$. In altre parole, si cerca di minimizzare l'errore introdotto nell'approssimare la distribuzione reale $P$ con il modello $Q$.

### Proprietà Chiave

- **Non-negatività:** La divergenza $D_{\text{KL}}(P \parallel Q)$ è sempre maggiore o uguale a zero. Questo valore è pari a zero solo quando le distribuzioni $P$ e $Q$ sono identiche, indicando una perfetta corrispondenza tra il modello e la distribuzione vera.
  
- **Asimmetria:** $D_{\text{KL}}(P \parallel Q) \neq D_{\text{KL}}(Q \parallel P)$. Questa proprietà mostra che la "distanza" percepita tra $P$ e $Q$ dipende dalla direzione in cui è misurata. La divergenza di $Q$ rispetto a $P$ non è la stessa della divergenza di $P$ rispetto a $Q$.

### Selezione dei Modelli Statistici

Nel contesto della selezione dei modelli statistici, l'obiettivo principale è scegliere il modello $Q$ che minimizzi la divergenza $D_{\text{KL}}(P \parallel Q)$ rispetto alla distribuzione reale $P$ dei dati. Tuttavia, poiché la distribuzione vera $P$ è spesso sconosciuta o non direttamente osservabile, non è possibile calcolare la divergenza $D_{\text{KL}}$ in modo diretto.

Per superare questa limitazione, i ricercatori e gli statistici utilizzano criteri approssimativi per stimare indirettamente la divergenza $D_{\text{KL}}$. Questi criteri considerano sia la bontà di adattamento del modello ai dati osservati sia la complessità del modello stesso, cercando un equilibrio tra accuratezza e parsimonia. Nel capitolo successivo, esploreremo questi criteri in dettaglio e discuteremo come vengono utilizzati per valutare e selezionare i modelli statistici migliori.

## Riflessioni Conclusive

In questo capitolo, abbiamo esaminato il concetto di entropia, evidenziando il suo ruolo fondamentale nel quantificare l'incertezza all'interno delle distribuzioni di probabilità. Abbiamo anche affrontato la questione di come l'entropia possa essere impiegata per valutare la "distanza" tra un modello teorico e i dati reali. A tale scopo, abbiamo introdotto la divergenza $\mathbb{KL}$, una misura che quantifica le discrepanze tra due distribuzioni di probabilità. 

Nel capitolo successivo, approfondiremo ulteriormente il tema della divergenza $\mathbb{KL}$. Esploreremo come questo strumento possa essere utilizzato per confrontare modelli teorici con dati empirici e ci concentreremo su come possa fornirci una comprensione più dettagliata dell'adattamento di un modello alla realtà che intende rappresentare. Questa esplorazione ci permetterà di valutare più accuratamente la validità e la generalizzabilità dei modelli scientifici nel loro tentativo di catturare e interpretare la complessità dei fenomeni oggetto di studio.

## Esercizi

::: {#exr-entropy-1}
Cosideriamo due distribuzioni di probabilità discrete, $p$ e $q$:

```
p = np.array([0.2, 0.5, 0.3])
q = np.array([0.1, 0.2, 0.7])
```

Si calcoli l'entropia di $p$, l'entropia incrociata tra $p$ e $q$, la divergenza di Kullback-Leibler da $p$ a $q$.

Si consideri `q = np.array([0.2, 0.55, 0.25])` e si calcoli di nuovo a divergenza di Kullback-Leibler da $p$ a $q$. Si confronti con il risultato precedente e si interpreti.
:::

::: {#exr-entropy-2}
Sia $p$ una distribuzione binomiale di parametri $\theta = 0.2$ e $n = 5$. Sia $q_1$ una approssimazione a $p$: `q1 = np.array([0.46, 0.42, 0.10, 0.01, 0.01])`. Sia $q_2$ una distribuzione uniforme: `q2 = [0.2] * 5`. Si calcoli la divergenza $\mathbb{KL}$ di $q_1$ da $p$ e da $q_2$ da $p$ e si interpretino i risultati.
:::

::: {#exr-entropy-3}
La Divergenza $\mathbb{KL}$ è spesso paragonata a una "distanza" tra due distribuzioni di probabilità, ma è fondamentale capire che non è simmetrica. Questo significa che la misura di quanto $p$ è diversa da $q$ non è la stessa di quanto $q$ è diversa da $p$. Questa asimmetria riflette la differenza nella perdita di informazione quando si sostituisce una distribuzione con l'altra.

Per le seguenti distribuzioni

```
p = np.array([0.01, 0.99])
q = np.array([0.7, 0.3])
```

si calcoli l'entropia di p, l'entropia incrociata da p a q, la divergenza KL da p a q, l'entropia di q, l'entropia incrociata da q a p, e la divergenza KL da q a p. Si commenti.
:::

## Informazioni sull'Ambiente di Sviluppo {.unnumbered}

In [12]:
%load_ext watermark
%watermark -n -u -v -iv -w -m

Last updated: Fri Jul 26 2024

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

numpy     : 1.26.4
matplotlib: 3.9.1
scipy     : 1.14.0
pandas    : 2.2.2
arviz     : 0.18.0

Watermark: 2.4.3

